# COMP7095 - Big Data Management

## Lab 6: Spark MLLib

### Introduction
Spark's machine learning library (MLLib) is used to make practical machine learning scalable and easy. In this lab, we will learn how to use MLLib to do a simple prediction.

Note that the data file, `heart.csv`, is originally from https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/heart/

### Creating DataFrame from Data File
Import the required packages and create a SQL context

In [ ]:
# Install pyspark
!pip install pyspark

import pyspark
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.types import *

spark = SparkSession.builder.getOrCreate()

Create a schema and use the SQL context to load the data from the file - `heart.csv`.

In [ ]:
schema = StructType([
    StructField('age', FloatType()),       
    StructField('sex', FloatType()),     
    StructField('chest pain', FloatType()),       
    StructField('resting blood pressure', FloatType()),
    StructField('serum cholesterol', FloatType()),
    StructField('fasting blood sugar', FloatType()),
    StructField('resting electrocardiographic results', FloatType()), 
    StructField('maximum heart rate achieved', FloatType()),
    StructField('exercise induced angina', FloatType()),
    StructField('ST depression induced by exercise relative to rest', FloatType()),
    StructField('the slope of the peak exercise ST segment', FloatType()),
    StructField('number of major vessels', FloatType()),
    StructField('thal', FloatType()),
    StructField('last', IntegerType())
])


### Task 1:

Create dataframe from `heart.csv` according to the above schema and display the dataframe to a pandas table. Hint, you can assume that the file has been uploaded.

The dataset contains 13 attributes:
Attributes info
1. age       
2. sex       
3. chest pain type  (4 values)       
4. resting blood pressure  
5. serum cholestoral in mg/dl      
6. fasting blood sugar > 120 mg/dl       
7. resting electrocardiographic results  (values 0,1,2) 
8. maximum heart rate achieved  
9. exercise induced angina    
10. oldpeak = ST depression induced by exercise relative to rest   
11. the slope of the peak exercise ST segment     
12. number of major vessels (0-3) colored by flourosopy        
13. thal: 3 = normal; 6 = fixed defect; 7 = reversable defect     

Attributes types
- Real: 1,4,5,8,10,12
- Ordered:11,
- Binary: 2,6,9
- Nominal:7,3,13

### Task 2: Statistics of dataset
Please display the maximum and minimum values of resting blood pressure


### Prediction

The following is an extra step to add an ID to each row. That lets you find the patients' info easier.

In [ ]:
from pyspark.sql.functions import monotonically_increasing_id
df = df.withColumn('id', monotonically_increasing_id())

We need two datasets - training set and test set. We split the data into two sets with 70/30 split.

In [ ]:
training, test = df.randomSplit([0.7, 0.3])
training.cache()


### Task 3
Display sizes of training set and size of test set, respectively.

For machine learning and classification or regression problems we have:

- Feature matrix, including patient's age, blood sugar, etc. (col 1 to 13)
- A vector of labels indicating whether a patient has a heart problem.

To create the vector, we need to import VectorAssembler.

In [ ]:
!pip install -U numpy

from pyspark.ml.feature import VectorAssembler

We select the feature columns.

In [ ]:
features = ('age',       
'sex',         
'chest pain',           
'resting blood pressure',    
'serum cholesterol',       
'fasting blood sugar',         
'resting electrocardiographic results', 
'maximum heart rate achieved',  
'exercise induced angina',     
'ST depression induced by exercise relative to rest',  
'the slope of the peak exercise ST segment',     
'number of major vessels',
'thal')

Then, we create a vector using `VectorAssembler` with the selected feature columns. A new column named features will be added to the dataframe.

In [ ]:
vecAssembler = VectorAssembler(inputCols=features,outputCol='features')


Everything is ready. We can now create a logistic regression model and train it with the training set. It compute the coefficients and intercept that most nearly matches the results that we have in the `last` column.

We also build a pipeline with the transformer and use it to apply the transformation to the dataset.

In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

lr = LogisticRegression(labelCol='last', featuresCol='features', maxIter=10)
pipeline = Pipeline(stages=[vecAssembler, lr])
pipe_model = pipeline.fit(training)

Then, we pass the test set to the model to make a prediction.

In [ ]:
predict_test=pipe_model.transform(test)

Let's see the prediction result! You can find the result on the `prediction` column.

In [ ]:
predict_test.select('id', 'last', 'prediction').orderBy('id').toPandas()

### Correctness
To check the correctness of the prediction, we add a function and use it to compare the `last` column and `prediction` column.

In [ ]:
from pyspark.sql.functions import udf

isCorrect = udf(lambda a,b: a == b)
check = predict_test.withColumn('correct', isCorrect('last', 'prediction'))

check.toPandas()

Count the number of "correct".

In [ ]:
check.groupby('correct').count().toPandas()

### Task 4
Calculate the accuracy. The accuracy can be caculated by $${\rm accuracy}= \frac{\rm \# `correct = true'}{\rm total \# rows}$$ and display the result.


### After using Spark
In the end, we should stop the Spark by using the stop function.

In [ ]:
spark.stop()